In [1]:
import numpy as np
from utils import read_canonical_model, load_pc, visualize_icp_result

obj_name = 'drill' # drill or liq_container
num_pc = 4 # number of point clouds

source_pc = read_canonical_model(obj_name, base_path=None)

for i in range(num_pc):
    target_pc = load_pc(obj_name, i, base_path=None)
    break

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [7]:
from sklearn.neighbors import NearestNeighbors

def get_pose(init_R, init_P):
    init_T= np.zeros([4,4])
    init_T[:3,:3]= init_R
    init_T[:, -1]= np.hstack((init_P, np.array([1])))
    return init_T

def get_R_and_P(T):
    return T[:3,:3], T[:-1,-1]

def get_pc_centroid(pc):
    return pc.mean(axis=0)
    
def get_corres_points(target_pc_moved, source_pc):
    kn_obj=NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(target_pc_moved)
    dist, idx= kn_obj.kneighbors(source_pc)
    return np.sum(dist.flatten()), idx.flatten()

def rotate_pc(R, pc):
    return (R@ pc.T).T

In [18]:
import transforms3d as t3d
from tqdm import tqdm


number_of_yaw_splits= 4
yaw_split= (2*np.pi)/number_of_yaw_splits
max_iters= 1000
sampled_target_pc= target_pc[::10]
sampled_source_pc= source_pc[::10]
euclidean_dist= []
best_pose=[]

for ii in range(0, number_of_yaw_splits):
    yaw_val= yaw_split*ii
    print('\n\n###########################\n\n')
    print("YAW_VALUE :: ", yaw_val)
    init_R= t3d.euler.euler2mat(0,0, yaw_val)
    init_p=  get_pc_centroid(source_pc)- get_pc_centroid(rotate_pc(init_R,target_pc) )
    print("Initial R: \n",init_R)
    print("Initial P: ",init_p)
    init_T= get_pose(init_R, init_p)
    print("Initial Pose:\n ",init_T)
    

    moved_target_pc= rotate_pc(init_R, sampled_target_pc)+init_p
    prev_euclidean_dist= 0
    current_euclidean_dist=0
    accumulated_pose = init_T
    dist, corres_idxs = get_corres_points(moved_target_pc,sampled_source_pc)
    print(dist)
    dist_between_corresponding_pc_pts= np.linalg.norm(sampled_source_pc-moved_target_pc[corres_idxs],axis=1)
    
    filter_dist_threshold= 0.5
    filtered_idx = np.where(dist_between_corresponding_pc_pts < filter_dist_threshold)
    
    sampled_source_pc=sampled_source_pc[filtered_idx]
    moved_target_pc=moved_target_pc[corres_idxs][filtered_idx]
    for itr in tqdm(range(max_iters)):
        dist, corres_idxs = get_corres_points(moved_target_pc,sampled_source_pc)

        #Kabasch stuff
        src_centroid = get_pc_centroid(sampled_source_pc)
        tar_centroid = get_pc_centroid(moved_target_pc)
    
        centered_sampled_source_pc= sampled_source_pc - src_centroid
        centered_moved_target_pc= moved_target_pc- tar_centroid
    
        Q= np.matmul(centered_sampled_source_pc.transpose(), centered_moved_target_pc[corres_idxs])
        U, S, Vt = np.linalg.svd(Q, full_matrices=True)
        R= U@Vt
        p= src_centroid - rotate_pc(R, tar_centroid)

        pred_pose= get_pose(R,p)
        accumulated_pose= pred_pose@accumulated_pose

        moved_target_pc = rotate_pc( R, moved_target_pc) + p
        if np.abs(dist-prev_euclidean_dist) < 1e-10:
            best_dist= dist
            break
        prev_euclidean_dist = dist
    print(accumulated_pose, best_dist)
    euclidean_dist.append(best_dist)
    best_pose.append(accumulated_pose)


visualize_icp_result(source_pc, target_pc,best_pose[np.argmin(euclidean_dist)])



###########################


YAW_VALUE ::  0.0
Initial R: 
 [[ 1.  0.  0.]
 [ 0.  1.  0.]
 [-0.  0.  1.]]
Initial P:  [-0.58054619 -0.0134431  -0.0444727 ]
Initial Pose:
  [[ 1.          0.          0.         -0.58054619]
 [ 0.          1.          0.         -0.0134431 ]
 [-0.          0.          1.         -0.0444727 ]
 [ 0.          0.          0.          1.        ]]
58.986466887691705


  1%|▏         | 14/1000 [00:00<00:04, 244.37it/s]


[[ 0.54593535  0.76781114  0.33529189 -0.34407122]
 [-0.65175371  0.64066889 -0.40590699  0.39594481]
 [-0.52647099  0.00307124  0.85018755  0.26745378]
 [ 0.          0.          0.          1.        ]] 30.823011686799294


###########################


YAW_VALUE ::  1.5707963267948966
Initial R: 
 [[ 6.123234e-17 -1.000000e+00  0.000000e+00]
 [ 1.000000e+00  6.123234e-17  0.000000e+00]
 [-0.000000e+00  0.000000e+00  1.000000e+00]]
Initial P:  [ 0.0134431  -0.58054619 -0.0444727 ]
Initial Pose:
  [[ 6.12323400e-17 -1.00000000e+00  0.00000000e+00  1.34431000e-02]
 [ 1.00000000e+00  6.12323400e-17  0.00000000e+00 -5.80546193e-01]
 [-0.00000000e+00  0.00000000e+00  1.00000000e+00 -4.44727001e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
56.26765557558488


  2%|▏         | 15/1000 [00:00<00:03, 267.08it/s]


[[-0.47431601 -0.86513063 -0.16301325  0.29603772]
 [ 0.69615097 -0.48193048  0.53208725 -0.42835456]
 [-0.53888603  0.13889567  0.83084887  0.2733437 ]
 [ 0.          0.          0.          1.        ]] 29.150709749461345


###########################


YAW_VALUE ::  3.141592653589793
Initial R: 
 [[-1.0000000e+00 -1.2246468e-16  0.0000000e+00]
 [ 1.2246468e-16 -1.0000000e+00  0.0000000e+00]
 [-0.0000000e+00  0.0000000e+00  1.0000000e+00]]
Initial P:  [ 0.58054619  0.0134431  -0.0444727 ]
Initial Pose:
  [[-1.00000000e+00 -1.22464680e-16  0.00000000e+00  5.80546193e-01]
 [ 1.22464680e-16 -1.00000000e+00  0.00000000e+00  1.34431000e-02]
 [-0.00000000e+00  0.00000000e+00  1.00000000e+00 -4.44727001e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
57.194240123289525


  1%|▏         | 13/1000 [00:00<00:04, 236.79it/s]

[[-0.5631342  -0.74414436 -0.35934529  0.35531846]
 [ 0.61923064 -0.66794482  0.41279913 -0.3777121 ]
 [-0.54720497  0.00994369  0.83693957  0.28214317]
 [ 0.          0.          0.          1.        ]] 30.479983483035273


###########################


YAW_VALUE ::  4.71238898038469
Initial R: 
 [[-1.8369702e-16  1.0000000e+00 -0.0000000e+00]
 [-1.0000000e+00 -1.8369702e-16  0.0000000e+00]
 [-0.0000000e+00  0.0000000e+00  1.0000000e+00]]
Initial P:  [-0.0134431   0.58054619 -0.0444727 ]
Initial Pose:
  [[-1.83697020e-16  1.00000000e+00 -0.00000000e+00 -1.34431000e-02]
 [-1.00000000e+00 -1.83697020e-16  0.00000000e+00  5.80546193e-01]
 [-0.00000000e+00  0.00000000e+00  1.00000000e+00 -4.44727001e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


52.41570260537423


  2%|▏         | 15/1000 [00:00<00:03, 256.23it/s]

[[ 0.46917134  0.85968029  0.20205951 -0.29495304]
 [-0.70353383  0.50215055 -0.5028767   0.43165094]
 [-0.53377749  0.09377964  0.84040881  0.27308202]
 [ 0.          0.          0.          1.        ]] 26.8683687330237


In [20]:
euclidean_dist

[30.823011686799294, 29.150709749461345, 30.479983483035273, 26.8683687330237]

In [13]:
accumulated_pose

array([[ 0.54593535,  0.76781114,  0.33529189, -0.34407122],
       [-0.65175371,  0.64066889, -0.40590699,  0.39594481],
       [-0.52647099,  0.00307124,  0.85018755,  0.26745378],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])